<a href="https://colab.research.google.com/github/PatrickJahn/easv-ML-mini-project/blob/main/ML_exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ML Mini Project

# Spam Email Classification - Random Forest


## Loading and preparation of data

In [ ]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data"
column_names = [f'feature_{i}' for i in range(57)] + ['is_spam']
df = pd.read_csv(url, names=column_names)

# Preprocess the data
# Split the data into features and target
X = df.drop('is_spam', axis=1)
y = df['is_spam']

# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Feature Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Training and fine-tuning of a model

In [ ]:
# Import the Random Forest Model and model selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Initialize the Random Forest classifier
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
random_forest.fit(X_train, y_train)


# Define a grid of hyperparameters to test
param_grid = {
    'n_estimators': [100, 200, 300],  # Number of trees in the forest
    'max_features': ['auto', 'sqrt'],  # Number of features to consider at every split
    'max_depth': [10, 20, 30, None],  # Maximum number of levels in tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split a node
    'min_samples_leaf': [1, 2, 4],  # Minimum number of samples required at each leaf node
    'bootstrap': [True, False]  # Method of selecting samples for training each tree
}

# Initialize the grid search model
grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42),
                           param_grid=param_grid,
                           cv=3,
                           n_jobs=-1,
                           verbose=2)

# Fit the grid search model
grid_search.fit(X_train, y_train)

# Print the best parameters
print("Best parameters found: ", grid_search.best_params_)

# Use the best model for predictions
best_grid = grid_search.best_estimator_
predictions = best_grid.predict(X_test)

## Evaluation of the model

In [ ]:
# Import necessary modules for evaluation
from sklearn.metrics import classification_report, accuracy_score

# Make predictions
predictions = random_forest.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, predictions))
print(classification_report(y_test, predictions))

# Evaluate the best model with fine tuning
print("Accuracy after fine-tuning:", accuracy_score(y_test, predictions))
print(classification_report(y_test, predictions))

# Income Level Prediction - MLPP


## Import

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer

## Loading and preparation of Data


In [ ]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
column_names = [
    'age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status',
    'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
    'hours_per_week', 'native_country', 'income'
]

# Load the data
data = pd.read_csv(url, names=column_names, na_values=' ?', sep=',\s', engine='python')

# Split the data into features and target label
X = data.drop('income', axis=1)
y = data['income'].apply(lambda x: 1 if x == '>50K' else 0)  # Convert income to binary

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Define categorical and numerical features for preprocessing
categorical_features = ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country']
numerical_features = ['age', 'fnlwgt', 'education_num', 'capital_gain', 'capital_loss', 'hours_per_week']

# Create preprocessors for numerical and categorical features
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

## Training and fine-tuning of a model

In [2]:
# Create an MLP model pipeline
mlp_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('mlpclassifier', MLPClassifier(hidden_layer_sizes=(50,), max_iter=500, random_state=42))
])

X_train_small, _, y_train_small, _ = train_test_split(X_train, y_train, train_size=0.3, random_state=42, stratify=y_train)

param_grid = {
    'mlpclassifier__hidden_layer_sizes': [(50,), (100,)],
    'mlpclassifier__activation': ['relu'],
    'mlpclassifier__solver': ['adam'],
    'mlpclassifier__alpha': [0.0001, 0.001]
}

grid_search = GridSearchCV(mlp_pipeline, param_grid, cv=3, scoring='accuracy', n_jobs=-1)
# Fine-tuning the MLP model
grid_search.fit(X_train_small, y_train_small)

# Print out the best parameters
print("Best parameters found: ", grid_search.best_params_)

NameError: name 'preprocessor' is not defined

## Evaluation of the model

In [2]:
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Age Classification from images - CNN

## Loading and preparation of data

In [3]:
# Import necessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib.image as mpimg


In [4]:
# Setup the path to the images and the attributes
PATH_TO_ATTR = 'Datasets/other/faces/train.csv'
PATH_TO_IMGS = 'Datasets/other/faces/Train'


# Then we check if there are the same amount of attributes and images
data = pd.read_csv(PATH_TO_ATTR)
images = os.listdir(PATH_TO_IMGS)

print(data.shape[0])
print(len(images))

FileNotFoundError: [Errno 2] No such file or directory: 'Datasets/other/faces/train.csv'

In [5]:
# Read the csv file to check out the attributes
data.describe()

NameError: name 'data' is not defined

In [6]:
# Check how many of each attribute class their is
groupedvalues = data.groupby('Class').count()
g = sns.barplot(x='Class', y='ID', data=groupedvalues)

NameError: name 'data' is not defined

In [7]:
# Replace the 3 Class values with a numeric value
data['Class'].replace(['YOUNG','MIDDLE','OLD'],[0,1,2],inplace=True)

# We want to make sure that the order of image classes is random
data.sample(frac=1)

data.head()

NameError: name 'data' is not defined

In [8]:
# Function to read the image an reformat it so all images are the same size and type
def readAndFormatImage(path):
      img = tf.io.read_file(path)
      img = tf.image.decode_jpeg(img, channels=3)
      img = tf.image.convert_image_dtype(img, dtype=tf.float32)
      img = tf.image.resize(img, (150, 150))
      return img

In [9]:
def load_data(image_path, label):
    img = readAndFormatImage(image_path)
    return (img, label)

In [10]:
# built the list of image paths and list of respective responses of the images
image_paths = []
for path in os.listdir(PATH_TO_IMGS):
    image_paths.append(PATH_TO_IMGS+"/"+path)
print(len(image_paths))

response_list = []

for i in image_paths:
    _,tail = os.path.split(i)
    data.loc
    response = data.loc[data['ID'] == tail]['Class'].values[0]
    response_list.append(response)
print(len(response_list))

FileNotFoundError: [Errno 2] No such file or directory: 'Datasets/other/faces/Train'

In [12]:
# split the dataset into train and test dataset
train_size = int(0.9*(len(image_paths)))
print(train_size)
test_size = int(0.1*(len(image_paths)))

train_set = tf.data.Dataset.from_tensor_slices((image_paths[:train_size], response_list[:train_size]))
test_set = tf.data.Dataset.from_tensor_slices((image_paths[test_size:], response_list[test_size:]))

0


NameError: name 'response_list' is not defined

In [13]:
train_set = (train_set
    .map(load_data, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(64)
    .prefetch(tf.data.AUTOTUNE)
)

test_set = (test_set
    .map(load_data, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(64)
    .prefetch(tf.data.AUTOTUNE)
)

NameError: name 'train_set' is not defined

## Training and fine-tuning of a model

In [ ]:
# build the layers of CNN model
from tensorflow.keras import layers,models

cnn_model = models.Sequential([
    layers.Conv2D(filters=30, kernel_size=(3, 3), activation='relu', input_shape=(150, 150, 3), padding = 'same'),
    layers.MaxPooling2D((2, 2)),


    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding = 'same'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(3, activation='softmax')
])

In [ ]:
cnn_model.summary()

In [ ]:
# compile the model
cnn_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# train the model
cnn_model.fit(train_set, epochs=10, validation_data=test_set)

## Evaluation of the model

In [14]:
# test accuracy
cnn_model.evaluate(test_set)

In [15]:
test_pred = cnn_model.predict(test_set)

NameError: name 'cnn_model' is not defined

In [ ]:
_labels = [np.argmax(item) for item in test_pred]
print("Test Predictions response sample:",y_labels[:10])

test_response = response_list[test_size:]
print("Test True response sample:", test_response[:10])

In [ ]:

Results = y_labels
Answers = test_response

# Define categories
categories = ['YOUNG', 'MIDDLE', 'OLD']

# Initialize dictionaries to store counts of correct and total answers for each category
correct_counts = {category: 0 for category in categories}
total_counts = {category: 0 for category in categories}

# Iterate through results and answers to count correct and total answers for each category
for result, answer in zip(Results, Answers):
    category_result = categories[result]
    category_answer = categories[answer]

    total_counts[category_answer] += 1
    if result == answer:
        correct_counts[category_result] += 1

# Calculate the percentage of correct answers for each category
percentages = [(correct_counts[category] / total_counts[category]) * 100 for category in categories]

# Plotting the bar graph
plt.bar(categories, percentages, color='skyblue')

# Adding labels and title
plt.xlabel('Category')
plt.ylabel('Percentage of Correct Answers')
plt.title('Percentage of Correct Answers for Each Category')

# Show the plot
plt.show()